# Assignment 16 - Logistic Regression
## Arturo Fornés Arvayo A01227071 April 25th, 2019

In [18]:
# install.packages("mlbench")

In [2]:
data(BreastCancer, package="mlbench")

In [3]:
# create copy
bc <- BreastCancer[complete.cases(BreastCancer), ] 

In [4]:
head(bc)

Id,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
1000025,5,1,1,1,2,1,3,1,1,benign
1002945,5,4,4,5,7,10,3,2,1,benign
1015425,3,1,1,1,2,2,3,1,1,benign
1016277,6,8,8,1,3,4,3,7,1,benign
1017023,4,1,1,3,2,1,3,1,1,benign
1017122,8,10,10,8,7,10,9,7,1,malignant


In [5]:
str(bc)

'data.frame':	683 obs. of  11 variables:
 $ Id             : chr  "1000025" "1002945" "1015425" "1016277" ...
 $ Cl.thickness   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 5 5 3 6 4 8 1 2 2 4 ...
 $ Cell.size      : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 1 1 2 ...
 $ Cell.shape     : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 2 1 1 ...
 $ Marg.adhesion  : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 5 1 1 3 8 1 1 1 1 ...
 $ Epith.c.size   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 2 7 2 3 2 7 2 2 2 2 ...
 $ Bare.nuclei    : Factor w/ 10 levels "1","2","3","4",..: 1 10 2 4 1 10 10 1 1 1 ...
 $ Bl.cromatin    : Factor w/ 10 levels "1","2","3","4",..: 3 3 3 3 3 9 3 3 1 2 ...
 $ Normal.nucleoli: Factor w/ 10 levels "1","2","3","4",..: 1 2 1 7 1 7 1 1 1 1 ...
 $ Mitoses        : Factor w/ 9 levels "1","2","3","4",..: 1 1 1 1 1 1 1 1 5 1 ...
 $ Class          : Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 2 1 1 1 1 ...


In [6]:
glm(Class ~ Cell.shape, family="binomial", data = bc)


Call:  glm(formula = Class ~ Cell.shape, family = "binomial", data = bc)

Coefficients:
 (Intercept)  Cell.shape.L  Cell.shape.Q  Cell.shape.C  Cell.shape^4  
       4.189        20.911         6.848         5.763        -1.267  
Cell.shape^5  Cell.shape^6  Cell.shape^7  Cell.shape^8  Cell.shape^9  
      -4.439        -5.183        -3.013        -1.289        -0.860  

Degrees of Freedom: 682 Total (i.e. Null);  673 Residual
Null Deviance:	    884.4 
Residual Deviance: 243.6 	AIC: 263.6

In [7]:
bc <- bc[,-1]

In [8]:
for(i in 1:9) {
    bc[, i] <- as.numeric(as.character(bc[, i]))
}

In [9]:
bc$Class <- ifelse(bc$Class == "malignant", 1, 0)
bc$Class <- factor(bc$Class, levels = c(0, 1))

In [10]:
str(bc)

'data.frame':	683 obs. of  10 variables:
 $ Cl.thickness   : num  5 5 3 6 4 8 1 2 2 4 ...
 $ Cell.size      : num  1 4 1 8 1 10 1 1 1 2 ...
 $ Cell.shape     : num  1 4 1 8 1 10 1 2 1 1 ...
 $ Marg.adhesion  : num  1 5 1 1 3 8 1 1 1 1 ...
 $ Epith.c.size   : num  2 7 2 3 2 7 2 2 2 2 ...
 $ Bare.nuclei    : num  1 10 2 4 1 10 10 1 1 1 ...
 $ Bl.cromatin    : num  3 3 3 3 3 9 3 3 1 2 ...
 $ Normal.nucleoli: num  1 2 1 7 1 7 1 1 1 1 ...
 $ Mitoses        : num  1 1 1 1 1 1 1 1 5 1 ...
 $ Class          : Factor w/ 2 levels "0","1": 1 1 1 1 1 2 1 1 1 1 ...


In [11]:
head(bc)

Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
5,1,1,1,2,1,3,1,1,0
5,4,4,5,7,10,3,2,1,0
3,1,1,1,2,2,3,1,1,0
6,8,8,1,3,4,3,7,1,0
4,1,1,3,2,1,3,1,1,0
8,10,10,8,7,10,9,7,1,1


In [12]:
table(bc$Class)


  0   1 
444 239 

In [14]:
library(caret)

Loading required package: lattice
Loading required package: ggplot2


In [15]:
# define 'not in' func
'%ni%' <- Negate('%in%')
# prevents printing scientific notations.
options(scipen=999)

In [16]:
# Prep Training and Test data.
set.seed(100)
# 70% training data
trainDataIndex <- createDataPartition(bc$Class, p=0.7,
list = F)
trainData <- bc[trainDataIndex, ]
testData <- bc[-trainDataIndex, ]
# show proportions
table(trainData$Class)


  0   1 
311 168 

In [17]:
# Down Sample
set.seed(100)
down_train <- downSample(x = trainData[,
 colnames(trainData) %ni% "Class"],
 y = trainData$Class)
# show proportions
table(down_train$Class)


  0   1 
168 168 

In [19]:
# Build Logistic Model
logitmod <- glm(Class ~ Cl.thickness + Cell.size +
Cell.shape, family = "binomial", data=down_train)
# Show summary
summary(logitmod)


Call:
glm(formula = Class ~ Cl.thickness + Cell.size + Cell.shape, 
    family = "binomial", data = down_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.0716  -0.1534  -0.0316   0.0193   2.8420  

Coefficients:
             Estimate Std. Error z value        Pr(>|z|)    
(Intercept)   -8.8755     1.3285  -6.681 0.0000000000237 ***
Cl.thickness   0.7293     0.1768   4.125 0.0000370162955 ***
Cell.size      1.0803     0.3334   3.240         0.00119 ** 
Cell.shape     0.8571     0.2920   2.935         0.00334 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 465.795  on 335  degrees of freedom
Residual deviance:  70.378  on 332  degrees of freedom
AIC: 78.378

Number of Fisher Scoring iterations: 8


In [20]:
# Now use it to predict the response on testData
pred <- predict(logitmod, newdata = testData, type =
"response")

In [21]:
head(pred)

1           2           5           8          13          15 
0.035863571 0.925571615 0.017621937 0.009733832 0.641806266 0.999850252

In [22]:
# set probabilities 1 or 0
y_pred_num <- ifelse(pred > 0.5, 1, 0)
# set probabilities 1 or 0 as factor
y_pred <- factor(y_pred_num, levels=c(0, 1))

In [23]:
y_act <- ifelse(testData$Class == "malignant", 1, 0)

In [25]:
mean(y_pred == testData$Class)

[1] 0.9411765